In [1]:
import numpy as np
import pandas as pd

In [104]:
df = pd.read_csv("weather_classification_data.csv")
df.head()

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
0,14.0,73,9.5,82.0,partly cloudy,1010.82,2,Winter,3.5,inland,Rainy
1,39.0,96,8.5,71.0,partly cloudy,1011.43,7,Spring,10.0,inland,Cloudy
2,30.0,64,7.0,16.0,clear,1018.72,5,Spring,5.5,mountain,Sunny
3,38.0,83,1.5,82.0,clear,1026.25,7,Spring,1.0,coastal,Sunny
4,27.0,74,17.0,66.0,overcast,990.67,1,Winter,2.5,mountain,Rainy


In [106]:
print(df['Cloud Cover'].unique())
print(df['Location'].unique())
print(df.isnull().sum())

['partly cloudy' 'clear' 'overcast' 'cloudy']
['inland' 'mountain' 'coastal']
Temperature             0
Humidity                0
Wind Speed              0
Precipitation (%)       0
Cloud Cover             0
Atmospheric Pressure    0
UV Index                0
Season                  0
Visibility (km)         0
Location                0
Weather Type            0
dtype: int64


In [108]:
df['Weather Type'].unique()

array(['Rainy', 'Cloudy', 'Sunny', 'Snowy'], dtype=object)

In [116]:
df.loc[df['Weather Type'] == 'Rainy'] = 0
df.loc[df['Weather Type'] == 'Cloudy'] = 1
df.loc[df['Weather Type'] == 'Sunny'] = 2
df.loc[df['Weather Type'] == 'Snowy'] = 3

In [118]:
df['Weather Type'] = df['Weather Type'].astype('int64')

df.dtypes

Temperature             float64
Humidity                  int64
Wind Speed              float64
Precipitation (%)       float64
Cloud Cover              object
Atmospheric Pressure    float64
UV Index                  int64
Season                   object
Visibility (km)         float64
Location                 object
Weather Type              int64
dtype: object

In [120]:
y = df['Weather Type']
df.drop(['Weather Type'], axis=1, inplace=True)

In [122]:
#one hot encoder
from sklearn.preprocessing import OneHotEncoder

cat_col = df.select_dtypes(include=['object']).columns.tolist()

ohe = OneHotEncoder(sparse_output=False)
encoded_data = ohe.fit_transform(df[cat_col])
df_encoded = pd.DataFrame(encoded_data, columns=ohe.get_feature_names_out(cat_col))

new_df = pd.concat([df, df_encoded], axis=1)
new_df.drop(cat_col, axis=1, inplace=True)

new_df.head()

,Temperature,Humidity,Wind Speed,Precipitation (%),Atmospheric Pressure,UV Index,Visibility (km),Cloud Cover_0,Cloud Cover_1,Cloud Cover_2,Cloud Cover_3,Season_0,Season_1,Season_2,Season_3,Location_0,Location_1,Location_2,Location_3
0,0.0,0,0.0,0.0,0.0,0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1.0,1,1.0,1.0,1.0,1,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2.0,2,2.0,2.0,2.0,2,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,2.0,2,2.0,2.0,2.0,2,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0,0.0,0.0,0.0,0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [124]:
new_df.dtypes
#all values are numbers-->good no other prep needed

Temperature             float64
Humidity                  int64
Wind Speed              float64
Precipitation (%)       float64
Atmospheric Pressure    float64
UV Index                  int64
Visibility (km)         float64
Cloud Cover_0           float64
Cloud Cover_1           float64
Cloud Cover_2           float64
Cloud Cover_3           float64
Season_0                float64
Season_1                float64
Season_2                float64
Season_3                float64
Location_0              float64
Location_1              float64
Location_2              float64
Location_3              float64
dtype: object

In [126]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(new_df, y, test_size=0.2, random_state=42)

print(X_train)
print(y_test)

       Temperature  Humidity  Wind Speed  Precipitation (%)  \
3958           0.0         0         0.0                0.0   
2239           0.0         0         0.0                0.0   
3608           2.0         2         2.0                2.0   
6848           0.0         0         0.0                0.0   
6119           1.0         1         1.0                1.0   
...            ...       ...         ...                ...   
11964          0.0         0         0.0                0.0   
5191           0.0         0         0.0                0.0   
5390           1.0         1         1.0                1.0   
860            1.0         1         1.0                1.0   
7270           0.0         0         0.0                0.0   

       Atmospheric Pressure  UV Index  Visibility (km)  Cloud Cover_0  \
3958                    0.0         0              0.0            1.0   
2239                    0.0         0              0.0            1.0   
3608                    

In [128]:
!pip3 install xgboost
import xgboost as xgb
from xgboost import XGBClassifier

In [129]:
model = xgb.XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [130]:
y_pred = model.predict(X_test)

In [131]:
y_test

4111     2
10607    2
7372     2
11786    3
12227    2
        ..
2543     0
96       0
2474     3
2522     3
3393     1
Name: Weather Type, Length: 2640, dtype: int64

In [133]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

1.0